# Kafka Producer Example - Sending JSON Messages

For an introduction to [Kafka](https://kafka.apache.org/), you may want to read some of the main [concepts](https://kafka.apache.org/documentation/#intro_concepts_and_terms).  An **event** records the fact that "something happened". An event has a key, value, timestamp, and optional metadata headers. **Producers** are those client applications that publish (write) events to Kafka, and **consumers** are those that subscribe to (read and process) these events.

This is an example of how to write a very simple Kafka [producer](https://kafka-python.readthedocs.io/en/master/apidoc/KafkaProducer.html) using the kafka-python library. This example connects to a Kafka broker configured with the `SASL_PLAIN` security protocol. It sends simple JSON formatted strings on a kafka **topic**.

You can also create and run consumers in a separate notebook such as [2_kafka_consumer_print.ipynb](./2_kafka_consumer_print.ipynb) to retrieve these events.

For further reading, visit the [documentation](https://kafka.apache.org/documentation/) for Kafka and for [kafka-python](https://kafka-python.readthedocs.io/)


## Dependencies

- [kafka-python](https://pypi.org/project/kafka-python/) Python client for the Apache Kafka distributed stream processing system. kafka-python is designed to function much like the official java client, with a sprinkling of pythonic interfaces (e.g., consumer iterators).


In [ ]:
!pip install -r requirements.txt

## Connection Information

Generally, much of your connection information (servers, username,  password) will be injected as environment variables.  This prevents a user from uploading private information to source control.

#### Expected Environment Variables
- `KAFKA_BOOTSTRAP_SERVER` location of the Kafka Bootstrap Server.  e.g. 'abc.xyz.kafka.rhcloud.com:443'
- `KAFKA_USERNAME` SASL username or client ID e.g. 'srvc-acct-1234-5678-abcd-efgj-12345678abcd'
- `KAFKA_PASSWORD` SASL password or client secret. e.g. 'abcd1234-5678-abcd-efgj-12345678abcd'


In [ ]:
import os

# location of the Kafka Bootstrap Server loaded from the environment variable.
# e.g. 'my-kafka-bootstrap.namespace.svc.cluster.local:9092'
KAFKA_BOOTSTRAP_SERVER = os.environ.get('KAFKA_BOOTSTRAP_SERVER')

# SASL settings.  Defaults to SASL_SSL/PLAIN.
# No auth would be PLAINTEXT/''
KAFKA_SECURITY_PROTOCOL = os.environ.get('KAFKA_SECURITY_PROTOCOL', 'SASL_SSL')
KAFKA_SASL_MECHANISM = os.environ.get('KAFKA_SASL_MECHANISM', 'PLAIN')

# SASL username or client ID loaded from the environment variable
KAFKA_USERNAME = os.environ.get('KAFKA_USERNAME')

# SASL password or client secret loaded from the environment variable
KAFKA_PASSWORD = os.environ.get('KAFKA_PASSWORD')

# Name of the topic for the producer to send messages.  
# Consumers will listen to this topic for events.
KAFKA_TOPIC = os.environ.get('KAFKA_TOPIC') or 'notebook-test'


## Creating the Producer

This function will create a producer that connects to the Kafka server and send messages to server set by variable `KAFKA_BOOTSTRAP_SERVER` on the topic set by variable `KAFKA_TOPIC`.  While this is sending messages, a consumer can read and process the events.

In [ ]:
import json
import time
from kafka import KafkaProducer


def produce_messages(start=1, end=100, delay=1):
    """Sends a number of messages in JSON format '{"txt": "hello 1"}'

    Keyword arguments:
    start -- start number (default 0)
    end -- last number to send (default 100)
    delay -- number of seconds between messages (default 1)
    """

    # create the producer
    producer = KafkaProducer(bootstrap_servers=[KAFKA_BOOTSTRAP_SERVER],
                             security_protocol=KAFKA_SECURITY_PROTOCOL,
                             sasl_mechanism=KAFKA_SASL_MECHANISM,
                             sasl_plain_username=KAFKA_USERNAME,
                             sasl_plain_password=KAFKA_PASSWORD,
                             api_version_auto_timeout_ms=30000,
                             max_block_ms=900000,
                             request_timeout_ms=450000,
                             acks='all')

    # send messages
    for x in range(start, end+1):
        time.sleep(delay)
        jsonpayload = json.dumps({'txt': f'hello {x}'})
        print(f'sending {jsonpayload}')
        producer.send(KAFKA_TOPIC, jsonpayload.encode('utf-8'))

    producer.flush()  # Important, especially if message size is small


## Send Messages

Starts the message loop.  After sending the number of messages, disconnects.


In [ ]:
# send 1000 messages with a 2 second delay
produce_messages(1, 1000, 2)
